In [17]:
from aiida.backends.utils import load_dbenv, is_dbenv_loaded
if not is_dbenv_loaded():
    load_dbenv(profile='test_dj1') # Insert the name of the test profile!
from age.utils import create_tree
from age.entities import get_basket
from age.rules import UpdateRule, RuleSequence, MODES

from aiida.common.links import  LinkType
from aiida.orm.querybuilder import QueryBuilder
from aiida.orm.node import Node
from aiida.orm.data import Data
from aiida.orm.group import Group
from aiida.orm.calculation.work import WorkCalculation

from aiida.utils.ascii_vis import draw_children

from datetime import datetime
import numpy as np

# The AiiDA Graph Explorer (AGE)
## Introduction
The Aiida Graph Explorer is a tool that allows to query the AiiDA Graph. For simple (and not-so-simple) queries you can already use the QueryBuilder functionality.
The functionality shown here can help targeting recursive queries and operations that can be described as *Update rules*. Some examples are:
 - Get all nodes that are connected (via any kind of link) to a given node.
 - If groups are defined as *adjacent* if they store the same node, get all connected groups of a certain group

## First Example: Getting all children of a Node

In the first example, I am creating an artifial tree. For every node, I create a fixed number of children. Each child becomes a parent to the same number of children, and so on...

In [18]:
# First I am creating nodes, by using a utility function.
# Recursively, up to level *DEPTH*, I am creating *NR_OF_CHILDREN* children
# for every node, and plotting the resulting tree:
# The number of layers I will create:
DEPTH = 4
# the branching at every level, i.e. the number of children per parent Node:
NR_OF_CHILDREN = 2

# Using a util function to create the tree:
descendants_dict = create_tree(DEPTH, NR_OF_CHILDREN)
# Using a visualizer within AiiDA!
draw_children(descendants_dict['parent'], dist=DEPTH)


                                           /-Calculation [1123]
                                /Data [1119]
                               |           \-Calculation [1124]
              /Calculation [1117]
             |                 |           /-Calculation [1125]
             |                  \Data [1120]
             |                             \-Calculation [1126]
-- /Data [1116]
             |                             /-Calculation [1127]
             |                  /Data [1121]
             |                 |           \-Calculation [1128]
              \Calculation [1118]
                               |           /-Calculation [1129]
                                \Data [1122]
                                           \-Calculation [1130]


Now, I want all the nodes that are exactly 2 levels down from my starting node. Obviously, I could use the QueryBuilder for this, but also a so-called update rule, that I run exactly 2 times on a basket of nodes. See for yourself:

In [19]:
# Using a utilty function to create a basket of results:
starting_basket = get_basket(node_ids=(descendants_dict['parent'].id,))
# I'm defining the QueryBuilder that defines an operation:
qb = QueryBuilder().append(Node, tag='n').append(Node, output_of='n')
# I'm instantiating a Rule instance with qb, in replace mode that will run for 2 iterations:
rule = UpdateRule(qb, mode=MODES.REPLACE, max_iterations=2)
# I run the rule on my starting basket:
res = rule.run(starting_basket)
# These are the results:
# My node-ids:
print(res.nodes) # prints the ids
# The actual entities:
print(list(res.nodes.get_entities()))

{1120,1121,1122,1119}
[<Data: uuid: 67136321-86df-4fbf-82a0-a2966a1bc281 (pk: 1119)>, <Data: uuid: f7bd9d69-453b-48c1-acbe-04497c886817 (pk: 1120)>, <Data: uuid: 2e7ddad8-5399-474c-a244-41e5f053fb70 (pk: 1121)>, <Data: uuid: 7338278a-d57a-4c11-a3ed-4ca6a56ffae6 (pk: 1122)>]


Let's review what happened here.
First, I got (via a utility function) of Basket of EntitySets. An EntitySet is a Set of AiiDA instances of the same top level of ORM-classes. I.e. you can have an EntitySet that contains 2 nodes, but no set that contains a group and a node.

Second, I define a QueryBuilder instance that will be given to the rule.
What happens is easy to describe: The first item in QueryBuilder path (the first append) will define the starting entities, and the last item in the QueryBuilder path (the last append) defines the resulting entities.

What if you want all the nodes that are up to 2 levels down? Not just the nodes exactly 2 levels down, but including the in-between nodes, and the starting node:

In [20]:
# Using a utilty function to create a Basket of results:
starting_basket = get_basket(node_ids=(descendants_dict['parent'].id,))
# I'm defining the QueryBuilder that defines an operation:
qb = QueryBuilder().append(Node, tag='n').append(Node, output_of='n')
# I'm instantiating a Rule instance with qb, in append mode that will run for 2 iterations:
rule = UpdateRule(qb, mode=MODES.APPEND, max_iterations=2)
# I run the rule on my starting basket:
res = rule.run(starting_basket)
# These are the results:
# My node-ids:
print(res.nodes) # prints the ids
# The actual entities:
print(list(res.nodes.get_entities()))

{1120,1121,1122,1116,1117,1118,1119}
[<Data: uuid: 3ddb1ed4-4096-44ff-9dec-e3635ea087be (pk: 1116)>, <Calculation: uuid: d8d33377-d6d2-4831-85f1-623156e0321a (pk: 1117)>, <Calculation: uuid: 6e20fc80-702f-4ffa-9d4e-54c1a44c2c4d (pk: 1118)>, <Data: uuid: 67136321-86df-4fbf-82a0-a2966a1bc281 (pk: 1119)>, <Data: uuid: f7bd9d69-453b-48c1-acbe-04497c886817 (pk: 1120)>, <Data: uuid: 2e7ddad8-5399-474c-a244-41e5f053fb70 (pk: 1121)>, <Data: uuid: 7338278a-d57a-4c11-a3ed-4ca6a56ffae6 (pk: 1122)>]


The only difference is that we changed from mode=MODES.REPLACE to mode=MODES.APPEND. These modes describe what happens at every iteration.

Next question: What happens if you run this rule not 1 or 2 times, but up to infinity? Try for yourself:

In [21]:
# Using a utilty function to create a Basket of results:
starting_basket = get_basket(node_ids=(descendants_dict['parent'].id,))
# I'm defining the QueryBuilder that defines an operation:
qb = QueryBuilder().append(Node, tag='n').append(Node, output_of='n')
# I'm instantiating a Rule instance with qb, in append mode,
# that will run until it has walked everywhere possible given by rule
rule = UpdateRule(qb, mode=MODES.APPEND, max_iterations=np.inf)
# I run the rule on my starting basket:
res = rule.run(starting_basket)
# These are the results:
# My node-ids:
print(res.nodes) # prints the ids
# The actual entities:
print(list(res.nodes.get_entities()))

{1116,1117,1118,1119,1120,1121,1122,1123,1124,1125,1126,1127,1128,1129,1130}
[<Data: uuid: 3ddb1ed4-4096-44ff-9dec-e3635ea087be (pk: 1116)>, <Calculation: uuid: d8d33377-d6d2-4831-85f1-623156e0321a (pk: 1117)>, <Calculation: uuid: 6e20fc80-702f-4ffa-9d4e-54c1a44c2c4d (pk: 1118)>, <Data: uuid: 67136321-86df-4fbf-82a0-a2966a1bc281 (pk: 1119)>, <Data: uuid: f7bd9d69-453b-48c1-acbe-04497c886817 (pk: 1120)>, <Data: uuid: 2e7ddad8-5399-474c-a244-41e5f053fb70 (pk: 1121)>, <Data: uuid: 7338278a-d57a-4c11-a3ed-4ca6a56ffae6 (pk: 1122)>, <Calculation: uuid: 5fd3a0f2-e0a0-4cab-9af5-5a489a6e092c (pk: 1123)>, <Calculation: uuid: 1dedaedf-8336-43a9-994f-18591d371558 (pk: 1124)>, <Calculation: uuid: 7f7e90a7-659d-48e1-8173-7beb7d3e78ad (pk: 1125)>, <Calculation: uuid: 25eca9e6-e4e5-4d15-9bc7-850c1fa4370c (pk: 1126)>, <Calculation: uuid: f157cf19-2719-42b2-bcc2-7ca5d0a990c8 (pk: 1127)>, <Calculation: uuid: 0434423e-c483-41fb-9c22-9e74801f8373 (pk: 1128)>, <Calculation: uuid: 7799fac0-f4a6-4810-b1ca-cf7

The rule will run as long as it can add new entities to its basket. Therefore, here we go into territory that the  QueryBuilder cannot capture, namely exploring the AiiDA graph on the fly.
But what will happen if there is a cycle, i.e. if we're not traversing a directed acyclic graph?

In [22]:
# Is the AGE deterred by cycles? No, it is not:
d = Data().store()
c = WorkCalculation().store()
# adding a loop, by having the data being both input to and returned by a workflow
c.add_link_from(d, link_type=LinkType.INPUT, label='lala')
d.add_link_from(c, link_type=LinkType.RETURN, label='lala')
qb = QueryBuilder().append(Node, tag='n').append(Node, output_of='n')
rule = UpdateRule(qb, mode=MODES.APPEND, max_iterations=np.inf)
es = get_basket(node_ids=(d.id,))
res = rule.run(es.copy())
print(res.nodes, {d.id, c.id})
print('I did {} iterations'.format(rule.get_iterations_done()))

({1131,1132}, set([1131, 1132]))
I did 2 iterations


As you can see, the AiiDA Graph Explorer does not go into infinite loops. This is because it keeps track of where it has been and only applies the rule to new entities entering the basket. When it has traversed the graph completely once, it will stop.

## Second Example: Groups and nodes

The AiiDA Graph Explorer is more general, and does not only work with nodes. It can also accepts instances of Group as entities. In the following example, I create a couple of groups and a couple of nodes, and interlink them such that you can walk from every node to any other node if you there is a group that both nodes belong to.

The first small rule that we define walks from a given node to any group that the node is a member of:

In [23]:
# Now, I create 4 nodes and 3 groups
nodes = [Node().store() for i in range(4)]
now = str(datetime.now())
groups = [Group(name='{}-{}'.format(now, i)).store() for i in range(3)]

# adding nodes 0 and 1 to group 0:
groups[0].add_nodes(nodes[:2])
# adding nodes 1 and 2 also to group 1
groups[1].add_nodes(nodes[1:3])

# adding nodes 2 and 3 to group 2
groups[2].add_nodes(nodes[2:4])

qb = QueryBuilder().append(Node, tag='n').append(Group, group_of='n')
rule = UpdateRule(qb, mode=MODES.APPEND, max_iterations=np.inf)
es = get_basket(node_ids=(nodes[1].id,))
print(rule.run(es.copy()))

  nodes_nodes: {}
  nodes: {1134}
  groups: {248,249}



What we need to walk from node to group to node is a *RuleSequence*, that is a way to execute several rules in series. This allows for very complicated **walks** in a graph. The RuleSequence that solves the little exercise of walking to all nodes that are connected via groups to recursive depth is as follows:

In [24]:
# I define rule1. That rule gets me from nodes to all groups that
# any node is a member of:
qb1 = QueryBuilder().append(Node, tag='n').append(Group, group_of='n')
rule1 = UpdateRule(qb1, mode=MODES.APPEND)

# I define rule2: That rule gets me from groups to all the nodes that are member
# of any group:
qb2 = QueryBuilder().append(Group, tag='g').append(Node, member_of='g')
rule2 = UpdateRule(qb2, mode=MODES.APPEND)

# I define the rule sequence, which means that rule1 and rule2 are applied in
# sequence, inside a loop that runs max_iterations times:
rs = RuleSequence((rule1, rule2), max_iterations=np.inf)
es = get_basket(node_ids=(nodes[0].id,))

# getting the results:
print(rs.run(es.copy()))

  nodes_nodes: {}
  nodes: {1133,1134,1135,1136}
  groups: {248,249,250}

